Other Ideas:
- Check whether images that are both in the test dataset are symetrically ranked highly
- Remove -1 images
- For ties in euclidean distance, select image that has a smaller time difference to reference image
- Use all data, also from kaggle
- Use ensemble 
- Use 2. place winner solution


In [1]:
import pandas as pd

In [2]:
model_name = 'seresnext101'
fold = 3
checkpoint = '17400'
algo = 'dist_global_avg'

In [3]:
df = pd.read_csv(f'WC_result/{model_name}_{fold}/out_{checkpoint}/{model_name}_sub_fold{fold}_{algo}.csv', header=None)


In [4]:
fp = '/home/pt-support/Humpback-Whale-Identification-1st-/WC_input/'
train = pd.read_csv(fp+'train.csv') 
image_to_id = dict(zip(train.Image, train.Id))

train = train.groupby('Id')['Image'].agg({lambda x: set(x)})
id_to_images = dict(zip(train.index, train.iloc[:, 0]))

In [5]:
id_to_images[image_to_id['PM-WWA-20160722-035.jpg']]

{'PM-WWA-20090612-092.jpg',
 'PM-WWA-20090811-188.jpg',
 'PM-WWA-20090813-127.jpg',
 'PM-WWA-20100716-156.jpg',
 'PM-WWA-20100723-468.jpg',
 'PM-WWA-20120617-014.jpg',
 'PM-WWA-20140627-075.jpg',
 'PM-WWA-20160722-035.jpg',
 'PM-WWA-20160726-060.jpg'}

In [6]:
id_to_images[image_to_id['PM-WWA-20160722-035.jpg']] - set(df.iloc[0])

{'PM-WWA-20090811-188.jpg',
 'PM-WWA-20090813-127.jpg',
 'PM-WWA-20100716-156.jpg'}

In [7]:
df[df.apply(lambda x: len(set(x)), axis=1) != 21]

,0,1,2,3,4,5,6,7,8,9,...,11,12,13,14,15,16,17,18,19,20


In [8]:
df.iloc[13].unique().shape

(21,)

In [9]:
list(id_to_images[image_to_id['PM-WWA-20160722-035.jpg']] - {'PM-WWA-20160722-035.jpg'})

['PM-WWA-20090612-092.jpg',
 'PM-WWA-20160726-060.jpg',
 'PM-WWA-20120617-014.jpg',
 'PM-WWA-20090813-127.jpg',
 'PM-WWA-20090811-188.jpg',
 'PM-WWA-20100723-468.jpg',
 'PM-WWA-20100716-156.jpg',
 'PM-WWA-20140627-075.jpg']

In [10]:
def get_train_img_class(row):
    ref = row[0]
    s = row[1:].copy()
    start = 0
    result = []
    for i, img in enumerate(row[1:]):
        try:
            imgs_same_class = list(id_to_images[image_to_id[img]])
            imgs_same_class = [e for e in imgs_same_class if e not in result]
            result.extend(imgs_same_class)                
        except:
            result.extend([img])
        
    return ([ref] + result)[:21]


In [11]:
df['new'] = df.apply(lambda x: get_train_img_class(x), axis=1)

In [12]:
df['new'].map(lambda x: len(set(x)))

0      21
1      21
2      21
3      21
4      21
       ..
803    21
804    21
805    21
806    21
807    21
Name: new, Length: 808, dtype: int64

In [13]:
df.tail(3)

,0,1,2,3,4,5,6,7,8,9,...,12,13,14,15,16,17,18,19,20,new
805,PM-WWA-20170801-061.jpg,PM-WWA-20170714-364.jpg,PM-WWA-20170715-021.jpg,PM-WWA-20180913-272.jpg,PM-WWA-20170719-203.jpg,PM-WWA-20120803-208.jpg,PM-WWA-20070619-018.jpg,PM-WWA-20170919-137.jpg,PM-WWA-20140813-020.jpg,PM-WWA-20180606-066.jpg,...,PM-WWA-20140822-301.jpg,PM-WWA-20100930-017.jpg,PM-WWA-20130815-399.jpg,PM-WWA-20060608-034.jpg,PM-WWA-20090425-378.jpg,PM-WWA-20060530-A003.jpg,PM-WWA-20120623-069.jpg,PM-WWA-20120804-082.jpg,PM-WWA-20100831-021.jpg,"[PM-WWA-20170801-061.jpg, PM-WWA-20170714-364...."
806,PM-WWA-20180713-173.jpg,PM-WWA-20160705-107.jpg,PM-WWA-20110615-009.jpg,PM-WWA-201006018-016.jpg,PM-WWA-20140608-153.jpg,PM-WWA-20090712-020.jpg,PM-WWA-20090704-126.jpg,PM-WWA-20110720-083.jpg,PM-WWA-20080727-097.jpg,PM-WWA-20110811-079.jpg,...,PM-WWA-20180806-015.jpg,PM-WWA-20080426-184.jpg,PM-WWA-20150726-173.jpg,PM-WWA-20170727-157.jpg,PM-WWA-20090623-011.jpg,PM-WWA-20090406-030.jpg,PM-WWA-20101023-049.jpg,PM-WWA-20080527-077.jpg,PM-WWA-20080824-084.jpg,"[PM-WWA-20180713-173.jpg, PM-WWA-20090712-020...."
807,PM-WWA-20170510-814.jpg,PM-WWA-20080513-136.jpg,PM-WWA-20050723-176.jpg,PM-WWA-20080728-026.jpg,PM-WWA-20150519-521.jpg,PM-WWA-20110520-124.jpg,PM-WWA-20180714-051.jpg,PM-WWA-20150526-063.jpg,PM-WWA-201006015-033.jpg,PM-WWA-20090614-084.jpg,...,PM-WWA-20070515-006.jpg,PM-WWA-20070901-047.jpg,PM-WWA-20150718-124.jpg,PM-WWA-20080819-105.jpg,PM-WWA-201006013-330.jpg,PM-WWA-20140626-100.jpg,PM-WWA-20120717-111.jpg,PM-WWA-20050704-005.jpg,PM-WWA-20160808-037.jpg,"[PM-WWA-20170510-814.jpg, PM-WWA-20080728-026...."


In [14]:
df['new'].apply(pd.Series).to_csv(f'WC_result/{model_name}_{fold}/out_{checkpoint}/postprocessing_{model_name}_sub_fold{fold}_{algo}.csv',
                                  header=None, index=False)